In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers
import tensorflow_probability as tfp

FFT_WIDTH = 80
input_shape = (FFT_WIDTH,3)

In [ ]:
#@keras_export('keras.layers.MultichannelInterpolate')
class MultichannelInterpolate(keras.layers.Layer):
    # TODO: Allow user to specify gaussian windows
    def __init__(self, harmonics):
        super(MultichannelInterpolate, self).__init__(trainable=False)
        self.harmonics = harmonics
        self.built = False

    def build(self, input_shape):
        dummy, self.clefs, self.buckets = input_shape
        super(MultichannelInterpolate, self).build(input_shape)

    def call(self, inputs, **kwargs):
        try:
            assert inputs.shape == (None, self.clefs, self.buckets)
        except:
            print("shape=(%d, %d, %d)" % (self.clefs, self.buckets, self.harmonics) + " inputs.shape=%s" % str(inputs.shape))
            #raise
        #inputs = tf.squeeze(inputs)
        inputs = tf.pad(inputs, tf.constant([[0,0],[0,0],[0,1]]), "CONSTANT") # Add padding to prevent interpolation errors
        inputs = inputs[..., tf.newaxis]                                # and a dummy 3rd dimension, so we can resize it like an image

        from scipy.signal.windows import gaussian
        from scipy.interpolate import interp1d
        import numpy as np

        windows = np.asarray([gaussian(2*self.buckets, 30)[self.buckets:], gaussian(self.buckets, 20), gaussian(2*self.buckets, 30)[:self.buckets]])
        windows = np.transpose(windows)

        output_list = []
        outputs = tf.zeros((self.harmonics,self.buckets), dtype=tf.float32)

        x = np.arange(0,self.buckets+1,1)
        #interpolator = interp1d(x,inputs)

        for h in range(self.harmonics):
            clef_multiplier = tf.constant(windows[h,:], dtype=tf.float32, shape=[1,3])

            interp_factor = 10.0 / (h+1)
            load = tf.image.resize(inputs, [self.clefs,(h+1)*8], method='bilinear', antialias=True, )[...,0]
            #load = interpolator(np.arange(0,80,interp_factor))[:,1:81]
            load = tf.linalg.matmul(clef_multiplier, load)

            try:
                if load.shape[2] < self.buckets:
                    load = tf.pad(load[:,0,:], tf.constant([[0,0],[0,self.buckets-load.shape[2]]]))
                else:
                    load = load[:,0,:self.buckets]
            except:
                print("h=%d, load=%s, clef_multiplier=%s, inputs=%s" % (h, load, clef_multiplier, inputs))
                raise

            output_list.append(load)
                #outputs[h,:len(load)] += load

        output = tf.stack(output_list)
        return tf.reshape(output, [-1, self.harmonics, self.buckets])

    def compute_output_shape(self, input_shape):
        return (self.harmonics, self.buckets)

In [ ]:
#calibrations = layers.Input(shape=input_shape)                      # Input is (freq, clef)

calibrations = layers.Input(shape=(3, FFT_WIDTH), name="calibrations")     # Have these fixed for now
x = MultichannelInterpolate(48)
x1 = x(calibrations)

inputs = layers.Input(shape=(FFT_WIDTH), name="main_input")                       # Input is (freq)

harmonic_extraction = layers.Dot(axes=(2,1))([x1, inputs])                         # Output is (bucket)

input_direct = layers.Dense(48, activation='relu')(inputs)
# pool = layers.AveragePooling1D(pool_size = FFT_WIDTH,         # Output is (scaled freq)
#         data_format="channels_first")(x2)

expanded = layers.Reshape((FFT_WIDTH, -1))(inputs)
c1 = layers.Conv1D(6, 5, strides=2, padding='same')(expanded)
c2 = layers.Conv1D(12, 5, strides=2, padding='same')(c1)
c3 = layers.Conv1D(24, 5, strides=2, padding='same')(c2)
c4 = layers.Conv1D(48, 5, strides=2, padding='same')(c3)
pool = layers.MaxPooling1D(5)(c4)
flatten_conv = layers.Flatten()(pool)

merge = layers.Concatenate()([harmonic_extraction, flatten_conv, input_direct])
flat = layers.Flatten()(merge)

out = layers.Dense(48, activation='sigmoid')(flat)

model = keras.Model(inputs=[calibrations, inputs], outputs=out)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer="adam")
model.summary()

In [ ]:
import pyaudio
import time
import numpy as np
from scipy.fft import fft, fftshift
from scipy import signal
import math

from matplotlib import pyplot as plt

pa = pyaudio.PyAudio()

LEN = 1764
OUTPUTS = 48

# def callback(in_data, frame_count, time_info, status):
#     data = np.fromstring(in_data, dtype=np.float32)
#
#     return (None, pyaudio.paContinue)


prompts = ["Play an open low E string", "Play an E string, 1st fret", "Play an E string, 2nd fret", "Play an E string, 3rd fret", "Play an E string, 4th fret",
           "Play an open A string", "Play an A string, 1st fret", "Play an A string, 2nd fret", "Play an A string, 3rd fret", "Play an A string, 4th fret",
           "Play an open D string", "Play a D string, 1st fret", "Play a D string, 2nd fret", "Play a D string, 3rd fret", "Play a D string, 4th fret",
           "Play an open G string", "Play a G string, 1st fret", "Play a G string, 2nd fret", "Play a G string, 3rd fret",
           "Play an open B string", "Play a B string, 1st fret", "Play a B string, 2nd fret", "Play a B string, 3rd fret", "Play a B string, 4th fret",
           "Play an open high E string", "Play an E string, 1st fret", "Play an E string, 2nd fret", "Play an E string, 3rd fret", "Play an E string, 4th fret"]

frames=[]
labels=[]

profile_name = input("Enter name to give guitar profile: ")
for i, p in enumerate(prompts):
    print(p)
    input("Press any key when ready")
    for j in range(3,0,-1):
        print(j)
        time.sleep(1)
    print("Now")
    l = np.zeros((OUTPUTS), dtype=float)
    l[i] = 1
    j = 0
    stream = pa.open(format=pyaudio.paFloat32, channels=1, rate=44100, input=True, output=False, frames_per_buffer=LEN)
    while stream.is_active() and j < 75:
        data = stream.read(LEN)
        data = np.frombuffer(data, dtype=np.float32)
        data = np.multiply(data, signal.windows.hamming(LEN))
        fft_arr = fft(data)[:int(LEN/2)]
        fft_arr = [math.sqrt(x.real**2 + x.imag**2) for x in fft_arr]
        fft_arr = [20 * math.log10(x) for x in fft_arr]
        frames.append(fft_arr[:100])
        labels.append(l)
        j+=1
    stream.close()

    # Plot note
    stacked = np.stack(frames[-75:], axis=0)
    avg = np.mean(stacked, axis=0)
    plt.plot(avg)
    plt.show()

frames = np.stack(frames, axis=0)
labels = np.stack(labels, axis=0)

np.save(profile_name + "_row" + str(int(LEN/2)) + ".npy", frames)
np.save(profile_name + "_labels" + "_row" + str(int(OUTPUTS)) + ".npy", labels)

# close PyAudio (7)
pa.terminate()

In [ ]:
import math
from matplotlib import pyplot as plt
import numpy as np

STANDARD_FREQ = 250

# frames_filename = input("Enter data npy file: ")
# labels_filename = input("Enter labels npy file: ")
frames_filename = "acoustic_deafened_row882.npy"
labels_filename = "acoustic_deafened_labels_row48.npy"
frames = np.load(frames_filename)
labels = np.load(labels_filename)

def note_to_freq(note):
    return 82.41 * math.pow(1.059463094, note)

standardized_bass = []
standardized_mid = []
standardized_treble = []

for f, l in zip(frames, labels):
    note = np.where(l == 1)[0]
    freq = note_to_freq(note)
    interp_factor = freq / STANDARD_FREQ
    interp = np.interp(np.linspace(0, interp_factor * 80, 80), np.linspace(0, 80, 80), f[:80])

    if (note < 10):
        standardized_bass.append(interp)
    elif(note < 19):
        standardized_mid.append(interp)
    else:
        standardized_treble.append(interp)

# Plot note
stacked = np.stack(standardized_bass, axis=0)
bass = np.mean(stacked, axis=0)
bass /= bass.max()
stacked = np.stack(standardized_mid, axis=0)
mid = np.mean(stacked, axis=0)
mid /= stacked.max()
stacked = np.stack(standardized_treble, axis=0)
treble = np.mean(stacked, axis=0)
treble /= stacked.max()
plt.plot(bass)
plt.title("Bass")
plt.show()
plt.plot(mid)
plt.title("Mid")
plt.show()
plt.plot(treble)
plt.title("Treble")
plt.show()

calibrations = np.stack([bass, mid, treble], axis=0)
np.save("calibrations.npy", calibrations)


In [ ]:
import tensorflow as tf
import numpy as np
from math import floor
import random
from sklearn.model_selection import train_test_split

frames_filename = "acoustic_oren_single_note_row80.npy"
labels_filename = "acoustic_oren_single_note_labels_row48.npy"
calibs_filename = "acoustic_oren_calib_row48.npy"
frames = np.load(frames_filename)
labels = np.load(labels_filename)
calibs = np.load(calibs_filename)

frames = np.concatenate((frames, np.load("acoustic_oren_row80.npy")), axis=0)
labels = np.concatenate((labels, np.load("acoustic_oren_labels_row48.npy")), axis=0)

data_size = frames.shape[0]
rind = list(range(data_size))
random.shuffle(rind)
frames = np.asarray([frames[i,:80] for i in rind])
labels = np.asarray([labels[i,:48] for i in rind])

frames_train, frames_test, labels_train, labels_test = train_test_split(frames, labels, test_size=0.2, random_state=1)
frames_train, frames_val, labels_train, labels_val = train_test_split(frames_train, labels_train, test_size=0.25, random_state=1)



calibs = np.expand_dims(calibs[1:],axis=0)
calibs_train = np.repeat(calibs,len(labels_train), axis=0)
calibs_valid = np.repeat(calibs, len(labels_val), axis=0)

model.fit([calibs_train, frames_train], labels_train, epochs=60, validation_data=([calibs_valid, frames_val], labels_val))


In [ ]:
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
import time


guesses = 0
false_positives = 0
total_labels = 0
false_negatives = 0
i = 0
for l, f in zip(labels_test, frames_test):
    before = time.time()
    p = model.predict([calibs, np.expand_dims(f, axis=0)])[0]
    #print("Time: %f" % (time.time() - before))

    func = K.function([model.layers[5].input, model.layers[0].input], model.layers[9].output)
    dot_out = func([calibs, np.expand_dims(f, axis=0)])

    conf = max(p)
    pred = np.where(p > 0.8)[0]
    correct = np.where(l == 1)[0]

    false_positives += len([x for x in pred if x not in correct])
    false_negatives += len([x for x in correct if x not in pred])
    guesses += len(pred)
    total_labels += len(correct)

    if i < 5:
        plt.plot(f)
        plt.title(correct)
        plt.show()
        # plt.plot(dot_out[0])
        # plt.show()
        plt.bar(np.arange(len(p)), p)
        plt.title(pred)
        plt.show()
        i+=1

print("False negative rate: " + str(false_negatives / total_labels))
print("False positive rate: " + str(false_positives / guesses))

In [ ]:
model.save("tf_model", save_format='tf')
model.summary()